In [2]:
import keras
import os
import pandas as pd
import numpy as np
import urllib
import random
import os
from sklearn import metrics
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from matplotlib import pyplot as plt


In [3]:
random.seed(42)


# AUC measure
def auc_score(y_true, y_pred):
    return metrics.roc_auc_score(y_true, y_pred)

def plot_ROC(fpr, tpr, roc_auc):
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.savefig("No-Glove-3layer.png")

# Read the data into a Pandas dataframe.
comments = pd.read_csv('attack_annotated_comments.tsv', sep = '\t', index_col = 0)
annotations = pd.read_csv('attack_annotations.tsv',  sep = '\t')

# Label a comment as an attack if over half of annotators did so.
# We can tinker with this threshold later.
labels = annotations.groupby('rev_id')['attack'].mean() > 0.5

# Join labels and comments
comments['attack'] = labels

# Preprocess the data -- remove newlines, tabs, quotes
# Something to consider: remove Wikipedia style markup (::'s and =='s)
comments['comment'] = comments['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
comments['comment'] = comments['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
comments['comment'] = comments['comment'].apply(lambda x: x.replace("`", " "))

In [4]:
# Grab the training, dev, and test data (3:1:1 split)
train_data = comments.loc[comments['split'] == 'train']
dev_data = comments.loc[comments['split'] == 'dev']
test_data = comments.loc[comments['split'] == 'test']


In [7]:
# The list of gold-standard labels for the data
train_labels = train_data["attack"].tolist()
dev_labels = dev_data["attack"].tolist()
test_labels = test_data["attack"].tolist()
# Put all the training data (comments) into a list
train_texts = train_data["comment"].tolist()
dev_texts = dev_data["comment"].tolist()
test_texts = test_data["comment"].tolist()

In [8]:
def randomly_sample(comments, labels):
    # grab all the attacks to see how many we need to match
    attack_indices = [i for i in range(len(comments)) if labels[i] == True]
    new_training = [comments[i] for i in attack_indices]
    new_labels = [labels[i] for i in attack_indices]
    
    # grab all the ones that are not attacks, shuffle them and select the same number of non-attacks
    non_attack_indices = [i for i in range(len(comments)) if labels[i] == False]
    random.shuffle(non_attack_indices)
    for i in range(len(attack_indices)):
        new_training.append(comments[i])
        new_labels.append(labels[i])
        
    # shuffle the training and labeling so that they're still matched 1:1
    shuf_indices = [i for i in range(len(new_training))]
    random.shuffle(shuf_indices)
    shuffled_training = [new_training[i] for i in shuf_indices]
    shuffled_labels = [new_labels[i] for i in shuf_indices]
    
    return shuffled_training, shuffled_labels
    

In [9]:
# randomly select non-attack data such that it creates an even split with the attack data
train_texts, train_labels = randomly_sample(train_texts, train_labels)
dev_texts, dev_labels = randomly_sample(dev_texts, dev_labels)
test_texts, test_labels = randomly_sample(test_texts, test_labels)
print(len(train_texts), len(train_labels))

840 840


In [10]:
# The word level tokenizer using only top 5000 words
tokenizer = keras.preprocessing.text.Tokenizer(num_words=5000)

In [11]:
# Fit it to the training data.
tokenizer.fit_on_texts(train_texts)

In [12]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

7987


In [13]:
dev_tokenizer =  keras.preprocessing.text.Tokenizer(num_words=5000)
dev_tokenizer.fit_on_texts(dev_texts)

test_tokenizer =  keras.preprocessing.text.Tokenizer(num_words=5000)
test_tokenizer.fit_on_texts(test_texts)

In [14]:
# Transform each comment in the training data to arrays of equal length
train_matrix = tokenizer.texts_to_matrix(train_texts)
dev_matrix = tokenizer.texts_to_matrix(dev_texts)
test_matrix = tokenizer.texts_to_matrix(test_texts)

In [15]:
# Dimensions of our training matrix
train_matrix.shape[0], train_matrix.shape[1]

(840, 5000)

In [16]:
# create an embedding layer of wordvec values
# assumes you have downloaded he glove.6b dataset here: http://nlp.stanford.edu/data/glove.6B.zip
embeddings_index = {}
f = open(os.path.join(str(os.getcwd())+"/glove.6B", 'glove.6B.200d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


Found 400000 word vectors.


In [17]:
# create a weight matrix for the embeddings

embedding_dim = 200
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [18]:
# Create custom skipgram embedding layer

embedding_layer = Embedding(vocab_size,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=train_matrix.shape[1],
                            trainable=False)

In [19]:
# Make a model.
model = Sequential()
# Add embedding layer.
model.add(Embedding(5000, 200, input_length=train_matrix.shape[1]))
model.add(Dense(200))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [20]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [21]:
# Train model
model.fit(train_matrix, train_labels,
          batch_size=100,
          epochs=4)

Epoch 1/4
840/840 [==============================] - 75s - loss: 6.9909 - acc: 0.5060      
Epoch 2/4
840/840 [==============================] - 63s - loss: 7.8383 - acc: 0.5083     

In [22]:

# Evaluate model
score, acc = model.evaluate(test_matrix, test_labels, batch_size=128)
print('\nAccuracy: %1.4f' % acc)
print('Score: %1.4f' % score)

296/296 [==============================] - 4s     

Accuracy: 0.5068
Score: 7.8635


In [23]:


y_pred = model.predict(test_matrix, batch_size=2)
roc = auc_score(test_labels, y_pred)
print('AUROC:{}'.format(roc))

AUROC:0.5


In [22]:
fpr, tpr, _ = metrics.roc_curve(test_labels, y_pred)
    
plot_ROC(fpr, tpr, roc)